#PEC 2

In [1]:
import numpy as np
import pandas as pd

from sklearn.linear_model import LogisticRegression

import joblib
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelBinarizer
from feature_engine.imputation import (
    MeanMedianImputer,
    AddMissingIndicator
)
from feature_engine.selection import DropFeatures

#to separate training and test
from sklearn.model_selection import train_test_split
#the model
from sklearn.naive_bayes import GaussianNB

In [2]:
df = pd.read_csv(r"C:\Users\isabe\Downloads\titanic(2).csv", sep=';')
df.head(10)

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,survived
0,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,2113375.00,B5,S,1
1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.55,C22 C26,S,1
2,1,"Allison, Miss. Helen Loraine",female,2.0000,1,2,113781,151.55,C22 C26,S,0
3,1,"Allison, Mr. Hudson Joshua Creighton",male,30.0000,1,2,113781,151.55,C22 C26,S,0
4,1,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",female,25.0000,1,2,113781,151.55,C22 C26,S,0
5,1,"Anderson, Mr. Harry",male,48.0000,0,0,19952,26.55,E12,S,1
6,1,"Andrews, Miss. Kornelia Theodosia",female,63.0000,1,0,13502,779583.00,D7,S,1
7,1,"Andrews, Mr. Thomas Jr",male,39.0000,0,0,112050,0.00,A36,S,0
8,1,"Appleton, Mrs. Edward Dale (Charlotte Lamson)",female,53.0000,2,0,11769,514792.00,C101,S,1
9,1,"Artagaveytia, Mr. Ramon",male,71.0000,0,0,PC 17609,495042.00,NaN,C,0


In [3]:
df.shape

(1308, 11)

In [4]:
df.dtypes

pclass        int64
name         object
sex          object
age         float64
sibsp         int64
parch         int64
ticket       object
fare        float64
cabin        object
embarked     object
survived      int64
dtype: object

#como sex tiene nulls...

In [5]:
df.info(verbose=True,show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1308 entries, 0 to 1307
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1308 non-null   int64  
 1   name      1308 non-null   object 
 2   sex       1308 non-null   object 
 3   age       1045 non-null   float64
 4   sibsp     1308 non-null   int64  
 5   parch     1308 non-null   int64  
 6   ticket    1308 non-null   object 
 7   fare      1308 non-null   float64
 8   cabin     295 non-null    object 
 9   embarked  1306 non-null   object 
 10  survived  1308 non-null   int64  
dtypes: float64(2), int64(4), object(5)
memory usage: 112.5+ KB


In [6]:
from sklearn.preprocessing import LabelEncoder
df['sex'] = LabelEncoder().fit_transform(df['sex'])
df['cabin'] = LabelEncoder().fit_transform(df['cabin'])
df.head(10)
df.info(verbose=True,show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1308 entries, 0 to 1307
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1308 non-null   int64  
 1   name      1308 non-null   object 
 2   sex       1308 non-null   int32  
 3   age       1045 non-null   float64
 4   sibsp     1308 non-null   int64  
 5   parch     1308 non-null   int64  
 6   ticket    1308 non-null   object 
 7   fare      1308 non-null   float64
 8   cabin     1308 non-null   int32  
 9   embarked  1306 non-null   object 
 10  survived  1308 non-null   int64  
dtypes: float64(2), int32(2), int64(4), object(3)
memory usage: 102.3+ KB


In [7]:
X, Y = df.loc[:, df.columns != 'survived'], df['survived']
X

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
0,1,"Allen, Miss. Elisabeth Walton",0,29.0000,0,0,24160,2113375.00,43,S
1,1,"Allison, Master. Hudson Trevor",1,0.9167,1,2,113781,151.55,79,S
2,1,"Allison, Miss. Helen Loraine",0,2.0000,1,2,113781,151.55,79,S
3,1,"Allison, Mr. Hudson Joshua Creighton",1,30.0000,1,2,113781,151.55,79,S
4,1,"Allison, Mrs. Hudson J C (Bessie Waldo Daniels)",0,25.0000,1,2,113781,151.55,79,S
...,...,...,...,...,...,...,...,...,...,...
1303,3,"Zabour, Miss. Hileni",0,14.5000,1,0,2665,144542.00,186,C
1304,3,"Zabour, Miss. Thamine",0,NaN,1,0,2665,144542.00,186,C
1305,3,"Zakarian, Mr. Mapriededer",1,26.5000,0,0,2656,7225.00,186,C
1306,3,"Zakarian, Mr. Ortin",1,27.0000,0,0,2670,7225.00,186,C


In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, #features
    Y, #labels
    test_size=0.3, #portion to test
    random_state=0 #seed definition)
)
X_train

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked
587,2,"Wells, Master. Ralph Lester",1,2.0000,1,1,29103,23.00,186,S
427,2,"Hamalainen, Master. Viljo",1,0.6667,1,1,250649,14.50,186,S
478,2,"Laroche, Miss. Louise",0,1.0000,1,2,SC/Paris 2123,415792.00,186,C
1228,3,"Strilic, Mr. Ivan",1,27.0000,0,0,315083,86625.00,186,S
453,2,"Hood, Mr. Ambrose Jr",1,21.0000,0,0,S.O.C. 14879,73.50,186,S
...,...,...,...,...,...,...,...,...,...,...
763,3,"Dean, Miss. Elizabeth Gladys ""Millvina""",0,0.1667,1,2,C.A. 2315,20575.00,186,S
835,3,"Guest, Mr. Robert",1,NaN,0,0,376563,8.05,186,S
1216,3,"Smyth, Miss. Julia",0,NaN,0,0,335432,77333.00,186,Q
559,2,"Sincock, Miss. Maude",0,20.0000,0,0,C.A. 33112,36.75,186,S


In [9]:
#CONFIGURATION

# numerical variables with NA in train set
NUMERICAL_VARS_WITH_NA = ['age']


# Drop features
DROP_FEATURES = ['name','embarked','parch', 'ticket', 'sibsp']

# the selected variables
FEATURES = ['pclass','sex', 'age', 'fare']


In [10]:
genero_pipe = Pipeline([

    # ===== IMPUTATION =====
    ('drop_features', DropFeatures(features_to_drop=DROP_FEATURES)),

    # impute numerical variables with the mean
    ('mean_imputation', MeanMedianImputer(
        imputation_method='mean', variables=NUMERICAL_VARS_WITH_NA
    )),
    ('LogReg', LogisticRegression())
   
   ])

In [11]:
genero_pipe.fit(X_train, Y_train.values.ravel())

Pipeline(steps=[('drop_features',
                 DropFeatures(features_to_drop=['name', 'embarked', 'parch',
                                                'ticket', 'sibsp'])),
                ('mean_imputation',
                 MeanMedianImputer(imputation_method='mean',
                                   variables=['age'])),
                ('LogReg', LogisticRegression())])

In [13]:
joblib.dump(genero_pipe, "MODELpec.pkl")

['MODELpec.pkl']

In [15]:
ejemplo = pd.DataFrame([[2,'ana', 1, 25, "1", "1", "33", 300, 22, "cor"]], columns = ["pclass", "name", "sex", "age", "sibsp", "parch", "ticket", "fare","cabin", "embarked"])
pred = genero_pipe.predict(ejemplo)
print(pred[0])

0
